In [33]:
# Core dependencies
!pip install langextract
!pip install python-dotenv


In [35]:
import textwrap
import langextract as lx

from IPython.display import display, HTML, Javascript


import os
from dotenv import load_dotenv

In [36]:
load_dotenv()

api_key = os.getenv("LANGEXTRACT_API_KEY")

In [48]:
import textwrap
import langextract as lx

# 1. Define prompt and extraction rules
prompt = textwrap.dedent("""\
Extract medical entities from the text, including:

- Patient names with roles
- Diseases (specify if chronic or acute)
- Medications, with dosage, frequency, duration
- Treatment relationships (which medication treats which disease)

Use the exact original text fragments for extraction.
Do not paraphrase or summarize.
Return a structured list of entities and their attributes in the order they appear.
If multiple medications or diseases are mentioned, list all with their details.
""")

# 2. Few-shot examples with medical data
examples = [
    lx.data.ExampleData(
        text="بیمار آقای رضا محمدی با سابقه فشار خون بالا و دیابت نوع ۲، به مدت یک ماه داروی آتنولول 50 میلی‌گرم روزانه و انسولین 20 واحد هر شب مصرف کرده است.",
        extractions=[
            lx.data.Extraction(
                extraction_class="Patient",
                extraction_text="آقای رضا محمدی",
                attributes={"role": "بیمار"}
            ),
            lx.data.Extraction(
                extraction_class="Disease",
                extraction_text="فشار خون بالا",
                attributes={"type": "مزمن"}
            ),
            lx.data.Extraction(
                extraction_class="Disease",
                extraction_text="دیابت نوع ۲",
                attributes={"type": "مزمن"}
            ),
            lx.data.Extraction(
                extraction_class="Medication",
                extraction_text="آتنولول",
                attributes={"dose": "50 میلی‌گرم", "duration": "یک ماه", "frequency": "روزانه"}
            ),
            lx.data.Extraction(
                extraction_class="Medication",
                extraction_text="انسولین",
                attributes={"dose": "20 واحد", "frequency": "هر شب"}
            ),
        ]
    )
]

# 3. Input text (longer medical note)
input_text = """
بیمار آقای علی رضایی، ۵۵ ساله، با سابقه بیماری قلبی و فشار خون بالا به کلینیک مراجعه کرده است.
وی به مدت ۶ ماه داروی آملودیپین 10 میلی‌گرم روزانه و آتورواستاتین 20 میلی‌گرم شبانه مصرف می‌کند.
همچنین، اخیراً دچار علائم دیابت نوع ۲ شده و انسولین 15 واحد صبح‌ها و 20 واحد شب‌ها تجویز شده است.
"""

# 4. Run extraction
result = lx.extract(
    text_or_documents=input_text,
    prompt_description=prompt,
    examples=examples,
    model_id="gemini-2.5-pro",
    api_key=api_key
)

print(result)

LangExtract: model=gemini-2.5-pro, current=289 chars, processed=289 chars:  [00:25]

✓ Extraction processing complete
✓ Extracted 8 entities (3 unique types)
  • Time: 25.92s
  • Speed: 11 chars/sec
  • Chunks: 1
AnnotatedDocument(extractions=[Extraction(extraction_class='Patient', extraction_text='آقای علی رضایی', char_interval=CharInterval(start_pos=7, end_pos=15), alignment_status=<AlignmentStatus.MATCH_LESSER: 'match_lesser'>, extraction_index=1, group_index=0, description=None, attributes={'role': 'بیمار'}), Extraction(extraction_class='Disease', extraction_text='بیماری قلبی', char_interval=CharInterval(start_pos=41, end_pos=52), alignment_status=<AlignmentStatus.MATCH_EXACT: 'match_exact'>, extraction_index=2, group_index=1, description=None, attributes={'type': 'مزمن'}), Extraction(extraction_class='Disease', extraction_text='فشار خون بالا', char_interval=CharInterval(start_pos=55, end_pos=68), alignment_status=<AlignmentStatus.MATCH_EXACT: 'match_exact'>, extraction_index=3, group_index=2, description=None, attributes={'type': 'مزمن'}), Extraction(extraction_cl

In [49]:
lx.io.save_annotated_documents([result], output_name="/content/extraction_results.jsonl")

LangExtract: Saving to extraction_results.jsonl: 1 docs [00:00, 629.68 docs/s]

✓ Saved 1 documents to extraction_results.jsonl


In [50]:

html = lx.visualize("/content/extraction_results.jsonl")
display(html)
# Inject CSS to change background color
display(HTML("""
<style>
    body {
        background-color: #f0f0f0 !important;
        color: black !important;
    }
    .lx-text, .lx-annotation, .annotated-text {
        color: black !important;
    }
</style>
"""))

LangExtract: Loading extraction_results.jsonl: 100%|██████████| 2.91k/2.91k [00:00<00:00, 6.56MB/s]

✓ Loaded 1 documents from extraction_results.jsonl
